# Download data

In [1]:
import requests
import os

In [2]:


if not os.path.exists('../data/diaries'):
    os.mkdir('../data/diaries')

def download_pdf(idLegislature, idPleno):
    url = f'http://www.congreso.es/public_oficiales/L{idLegislature}/CONG/DS/PL/DSCD-{idLegislature}-PL-{idPleno}.PDF'
    response = requests.get(url)
    if response.status_code == 200:
        with open(f'../data/diaries/DSCD-{idLegislature:02d}-PL-{idPleno:03d}.PDF', 'wb') as file:
            file.write(response.content)
        return url
    return None



consecutive_errors = 0
for legislature in range (10, 15):
    consecutive_errors = 0
    for pleno in range(1, 100):
        path = download_pdf(legislature, pleno)
        if path:
            consecutive_errors = 0
        else:            
            consecutive_errors = consecutive_errors + 1
            if consecutive_errors > 50:
                break

KeyboardInterrupt: 

In [8]:
url = "http://my-api/api/pdfconverter"
pdfsPath = '../data/diaries/'
txtPath = '../data/diaries-txts/'

if not os.path.exists(txtPath):
    os.mkdir(txtPath)

files = [f for f in os.listdir(pdfsPath) if os.path.isfile(os.path.join(pdfsPath, f))]

for file in files:
    files={ ('file', (file, open(f'{pdfsPath}{file}', 'rb'), 'application/octet-stream')) }
    response = requests.post(url, files=files)
    text = response.text
    with open(f'{txtPath}{file[:-4]}.txt', 'w') as f:
        f.write(text)

In [4]:
resp = requests.get("http://my-elasticsearch:9200")
print(resp.text)

{
  "name" : "my-elasticsearch",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "TrnE0Z3ZRh2tnvlxwDa-Yw",
  "version" : {
    "number" : "7.10.1",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "1c34507e66d7db1211f66f3513706fdf548736aa",
    "build_date" : "2020-12-05T01:00:33.671820Z",
    "build_snapshot" : false,
    "lucene_version" : "8.7.0",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}

